In [1]:
# Imports

from collections import defaultdict, Counter, deque
from functools import lru_cache
import itertools
from intervaltree import IntervalTree, Interval
import json
import numpy as np
import numpy.typing as npt
import re
from sortedcontainers import SortedList
from typing import Callable, TypeVar, Union, Optional


T = TypeVar('T')

# Helper functions

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

def split_csv_row(row: str) -> list[int]:
  return [int(x) for x in row.split(',')]

def split_int_row(row: str) -> list[int]:
  return [int(x) for x in row]

# Day 1

In [19]:
def parse_day1(data):
  elves = SortedList()
  current = 0
  for food in data:
    if not food:
      elves.add(current)
      current = 0
    else:
      current += int(food)
  return elves

data1 = parse_day1(data(1))

In [21]:
data1[-1]

71780

In [23]:
sum(data1[-3:])

212489

# Day 2

In [13]:
def parse2(data):
  result = Counter()
  for round in data:
    result[round] += 1
  return result
data2 = parse2(data(2))

In [11]:
score = {
  'A X': 3 + 1, # Rock Rock
  'A Y': 6 + 2, # Rock Paper
  'A Z': 0 + 3, # Rock Scissor
  'B X': 0 + 1, # Paper Rock
  'B Y': 3 + 2, # Paper Paper
  'B Z': 6 + 3, # Paper Scissor
  'C X': 6 + 1, # Scissor Rock
  'C Y': 0 + 2, # Scissor Paper
  'C Z': 3 + 3, # Scissor Scissor
}
sum([score[x]*data2[x] for x in data2])

13924

In [12]:
score = {
  'A X': 0 + 3, # Rock Lose Sciss
  'A Y': 3 + 1, # Rock Draw Rock
  'A Z': 6 + 2, # Rock Win Paper
  'B X': 0 + 1, # Paper Lose Rock
  'B Y': 3 + 2, # Paper Draw Paper
  'B Z': 6 + 3, # Paper Win Sciss
  'C X': 0 + 2, # Scissor Lose Paper
  'C Y': 3 + 3, # Scissor Draw Sciss
  'C Z': 6 + 1, # Scissor Win Rock
}
sum([score[x]*data2[x] for x in data2])

13448

# Day 3

In [29]:
data3 = data(3)

def item_priority(item):
  if ord(item) > 96:
    return ord(item) - 96
  return ord(item) - 38

In [31]:
def rucksack_items(rucksack):
  length = len(rucksack)//2
  return rucksack[:length], rucksack[length:]
  
def day3(data):
  sum = 0
  for x in data:
    a, b = rucksack_items(x)
    sum += item_priority(list(set(a).intersection(set(b)))[0])
  return sum

day3(data3)

7727

In [36]:
def day3_2(data):
  sum = 0
  for i in range(len(data)//3):
    a, b, c = data[i*3:i*3+3]
    common = set(a).intersection(set(b)).intersection(set(c))
    sum += item_priority(list(common)[0])
  return sum

day3_2(data3)

2609

# Day 4

In [43]:
data4 = data(4)

In [41]:
def has_envelop(a, b):
  return a[0] <= b[0] and a[1] >= b[1]

def day4(data, f):
  count = 0
  for i in data:
    a, b = [[int(x) for x in elf.split('-')] for elf in i.split(',')]
    count += f(a, b) or f(b, a)
  return count

day4(data4, has_envelop)

2

In [44]:
def has_overlap(a, b):
  return (a[0] <= b[0] and a[1] >= b[0]) or (a[0] >= b[0] and a[0] <= b[1])

day4(data4, has_overlap)

891

# Day 5

In [108]:
def make_move(positions, count, start, end):
  if count == 0:
    return
  item = positions[start].popleft()
  positions[end].appendleft(item)
  return make_move(positions, count-1, start, end)

def read_top(positions):
  return ''.join([positions[i][0] for i in range(len(positions))])

def day5(make_move):
  def parse5():
    with open('./data/day5.txt') as f:
      lines = f.readlines()
    positions = defaultdict(lambda: deque())
    line_length = len(lines[0])
    for n, line in enumerate(lines):
      for i in range(line_length//4):
        item = line[i*4:i*4+4].strip()
        if item == '1':
          return positions, lines[n+2:]
        if item:
          positions[i].append(re.match('\[([A-Z])\]', item)[1])
  positions, moves = parse5()
  for turn in moves:
    result = re.match('move (\d+) from (\d) to (\d)', turn)
    make_move(positions, int(result[1]), int(result[2])-1, int(result[3])-1)
  return read_top(positions)

print(day5(make_move))

RFFFWBPNS


In [109]:
def make_move_9001(positions, count, start, end):
  stack = deque()
  for _ in range(count):
    stack.appendleft(positions[start].popleft())
  positions[end].extendleft(stack)

print(day5(make_move_9001))

CQQBBJFCS


# Day 6

In [12]:
data6 = data(6)[0]

In [19]:
def day6(n):
  for i in range(len(data6)-n+1):
    if len(set(data6[i:i+n])) == n:
      return i + n

day6(4)

1262

In [15]:
day6(14)

3444

# Day 7

In [36]:
class Dir:
  def __init__(self, parent=None):
    self.parent = parent
    self.files = 0
    self.children = {}

def parse_command(line, cwd, ls, root):
  for item in ls:
    if item.startswith('dir'):
      name = item[4:]
      if not cwd.children.get(name):
        cwd.children[name] = Dir(cwd)
    else:
      cwd.files += int(item.split(' ')[0])

  if line.startswith('cd'):
    cwd = root if line == 'cd /' else cwd.parent if '..' in line else cwd.children[line.split(' ')[1]]
  return cwd

def parse7(data):
  root = Dir()
  cwd, ls = root, []
  for line in data:
    if line[0] == '$':
      cwd, ls = parse_command(line[2:], cwd, ls, root), []
    else:
      ls.append(line)
  if ls:
    parse_command('ls', cwd, ls, root)
  return root

data7 = parse7(data(7))

In [65]:
total_sizes = []
def calculate_total_sizes(root):
  total = root.files
  for child in root.children:
    total += calculate_total_sizes(root.children[child])
  total_sizes.append(total)
  return total
total = calculate_total_sizes(data7)

In [66]:
sum([x for x in total_sizes if x <= 100000])

1206825

In [69]:
target = 48729145 - (70000000 - 30000000)
for i in sorted(total_sizes):
  if i >= target:
    break
print(i)

9608311


# Day 8